In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Epilepsy"
cohort = "GSE29796"

# Input paths
in_trait_dir = "../../input/GEO/Epilepsy"
in_cohort_dir = "../../input/GEO/Epilepsy/GSE29796"

# Output paths
out_data_file = "../../output/preprocess/Epilepsy/GSE29796.csv"
out_gene_data_file = "../../output/preprocess/Epilepsy/gene_data/GSE29796.csv"
out_clinical_data_file = "../../output/preprocess/Epilepsy/clinical_data/GSE29796.csv"
json_path = "../../output/preprocess/Epilepsy/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Transcriptional Differences between Normal and Glioma-Derived Glial Progenitor Cells Identify a Core Set of Dysregulated Genes."
!Series_summary	"Glial progenitor cells (GPCs) of the adult human white matter, which express gangliosides recognized by monoclonal antibody A2B5, are a potential source of glial tumors of the brain. We used A2B5-based sorting to extract progenitor-like cells from a range of human glial tumors, that included low-grade glioma, oligodendroglioma, oligo-astrocytomas, anaplastic astrocytoma, and glioblastoma multiforme. The A2B5+ tumor cells proved tumorigenic upon orthotopic xenograft, and the tumors generated reflected the phenotypes of those from which they derived."
!Series_summary	"Expression profiling revealed that A2B5+ tumor progenitors expressed a cohort of genes by which they could be distinguished from A2B5+ GPCs isolated from normal adult white matter. Most of the genes differentially expressed by glioma-derived 

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the series title and summary, this dataset appears to contain gene expression data
# The study focuses on transcriptional differences between normal and glioma-derived glial progenitor cells
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# For trait (Epilepsy):
# In sample characteristics dictionary, key 1 contains 'pathology: epilepsy' which indicates trait information
trait_row = 1

# For age:
# There's no age information in the sample characteristics dictionary
age_row = None

# For gender:
# There's no gender information in the sample characteristics dictionary
gender_row = None

# 2.2 Data Type Conversion
# Function to convert trait values to binary (1 for Epilepsy, 0 for others)
def convert_trait(value):
    if value is None:
        return None
    # Extract the value after colon
    if isinstance(value, str) and ':' in value:
        value = value.split(':', 1)[1].strip().lower()
    elif isinstance(value, str):
        value = value.strip().lower()
    else:
        return None
    
    # Return 1 for Epilepsy, 0 for all other conditions (which appear to be various types of tumors)
    if value == 'epilepsy':
        return 1
    else:
        return 0

# Function for age conversion (not needed since age_row is None, but defining for completeness)
def convert_age(value):
    return None

# Function for gender conversion (not needed since gender_row is None, but defining for completeness)
def convert_gender(value):
    return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None
# Initial filtering on usability of the dataset
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is not None, we would normally proceed with clinical feature extraction
# However, since we don't have the actual sample-level clinical data, we'll skip this step
# The initial validation has already been saved correctly above
print("Clinical feature extraction skipped - actual sample data not available.")
print(f"Initial validation saved to {json_path}")


Clinical feature extraction skipped - actual sample data not available.
Initial validation saved to ../../output/preprocess/Epilepsy/cohort_info.json


### Step 3: Gene Data Extraction

In [4]:
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"SOFT file: {soft_file}")
print(f"Matrix file: {matrix_file}")

# Set gene availability flag
is_gene_available = True  # Initially assume gene data is available

# First check if the matrix file contains the expected marker
found_marker = False
try:
    with gzip.open(matrix_file, 'rt') as file:
        for line in file:
            if "!series_matrix_table_begin" in line:
                found_marker = True
                break
    
    if found_marker:
        print("Found the matrix table marker in the file.")
    else:
        print("Warning: Could not find '!series_matrix_table_begin' marker in the file.")
        
    # Try to extract gene data from the matrix file
    gene_data = get_genetic_data(matrix_file)
    
    if gene_data.shape[0] == 0:
        print("Warning: Extracted gene data has 0 rows.")
        is_gene_available = False
    else:
        print(f"Gene data shape: {gene_data.shape}")
        # Print the first 20 gene/probe identifiers
        print("First 20 gene/probe identifiers:")
        print(gene_data.index[:20].tolist())
        
except Exception as e:
    print(f"Error extracting gene data: {e}")
    is_gene_available = False
    
    # Try to diagnose the file format
    print("Examining file content to diagnose the issue:")
    try:
        with gzip.open(matrix_file, 'rt') as file:
            for i, line in enumerate(file):
                if i < 10:  # Print first 10 lines to diagnose
                    print(f"Line {i}: {line.strip()[:100]}...")  # Print first 100 chars of each line
                else:
                    break
    except Exception as e2:
        print(f"Error examining file: {e2}")

if not is_gene_available:
    print("Gene expression data could not be successfully extracted from this dataset.")


SOFT file: ../../input/GEO/Epilepsy/GSE29796/GSE29796_family.soft.gz
Matrix file: ../../input/GEO/Epilepsy/GSE29796/GSE29796_series_matrix.txt.gz
Found the matrix table marker in the file.


Gene data shape: (29385, 72)
First 20 gene/probe identifiers:
['1007_s_at', '1053_at', '117_at', '1294_at', '1405_i_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552263_at', '1552264_a_at', '1552266_at', '1552274_at', '1552275_s_at', '1552277_a_at', '1552281_at', '1552283_s_at', '1552286_at', '1552287_s_at', '1552291_at', '1552302_at']


### Step 4: Gene Identifier Review

In [5]:
# The identifiers appear to be Affymetrix probe IDs (e.g., '1007_s_at', '1053_at'), not human gene symbols
# These need to be mapped to gene symbols for proper biological interpretation

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Based on the preview, 'ID' appears to be the probe ID and 'Description' contains gene names
# Display more samples from the Description column to better understand the format
print("\nSample of Description column (first 5 rows):")
if 'Description' in gene_annotation.columns:
    for i in range(min(5, len(gene_annotation))):
        print(f"Row {i}: {gene_annotation['Description'].iloc[i]}")



Gene annotation preview:
Columns in gene annotation: ['ID', 'GB_ACC', 'SPOT_ID', 'Species Scientific Name', 'Annotation Date', 'Sequence Type', 'Sequence Source', 'Target Description', 'Representative Public ID', 'Gene Title', 'Gene Symbol', 'ENTREZ_GENE_ID', 'RefSeq Transcript ID', 'Gene Ontology Biological Process', 'Gene Ontology Cellular Component', 'Gene Ontology Molecular Function']
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Observe the gene annotation data to determine which columns to use for mapping
# From the preview, we can see that 'ID' contains the probe IDs (like '1007_s_at') which match the gene expression data
# 'Gene Symbol' contains the human gene symbols (like 'DDR1 /// MIR4640')

# 2. Extract the mapping between probe IDs and gene symbols
gene_mapping = get_gene_mapping(gene_annotation, 'ID', 'Gene Symbol')
print(f"Gene mapping shape: {gene_mapping.shape}")
print(f"Sample of gene mapping (first 5 rows):")
print(gene_mapping.head())

# 3. Apply the gene mapping to convert probe-level measurements to gene-level expression
# The apply_gene_mapping function handles the many-to-many relationships as described
gene_data = apply_gene_mapping(gene_data, gene_mapping)
print(f"Gene expression data after mapping: {gene_data.shape}")
print(f"First 10 genes in the mapped data:")
print(list(gene_data.index[:10]))

# Check if we have any genes in the result
if gene_data.shape[0] == 0:
    print("Warning: No genes were mapped! Check if the gene mapping process worked correctly.")
    is_gene_available = False
else:
    print(f"Successfully mapped probes to {gene_data.shape[0]} genes.")
    is_gene_available = True


Gene mapping shape: (45782, 2)
Sample of gene mapping (first 5 rows):
          ID              Gene
0  1007_s_at  DDR1 /// MIR4640
1    1053_at              RFC2
2     117_at             HSPA6
3     121_at              PAX8
4  1255_g_at            GUCA1A
Gene expression data after mapping: (15042, 72)
First 10 genes in the mapped data:
['A2M', 'A2M-AS1', 'A2MP1', 'AACS', 'AADAT', 'AAED1', 'AAGAB', 'AAK1', 'AAMP', 'AAR2']
Successfully mapped probes to 15042 genes.


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
try:
    # Make sure the directory exists
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    
    # Use the gene_data variable from the previous step (don't try to load it from file)
    print(f"Gene data shape before normalization: {gene_data.shape}")
    
    # Apply normalization to gene symbols
    normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
    print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
    
    # Save the normalized gene data
    normalized_gene_data.to_csv(out_gene_data_file)
    print(f"Normalized gene data saved to {out_gene_data_file}")
    
    # Use the normalized data for further processing
    gene_data = normalized_gene_data
    is_gene_available = True
except Exception as e:
    print(f"Error normalizing gene data: {e}")
    is_gene_available = False

# 2. Load clinical data - respecting the analysis from Step 2
# From Step 2, we determined:
# trait_row = None  # No Epilepsy data available
# age_row = None
# gender_row = None
is_trait_available = trait_row is not None

# Skip clinical feature extraction when trait_row is None
if is_trait_available:
    try:
        # Load the clinical data from file
        soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
        background_info, clinical_data = get_background_and_clinical_data(matrix_file)
        
        # Extract clinical features
        clinical_features = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            gender_row=gender_row,
            convert_gender=convert_gender,
            age_row=age_row,
            convert_age=convert_age
        )
        
        print(f"Extracted clinical data shape: {clinical_features.shape}")
        print("Preview of clinical data (first 5 samples):")
        print(clinical_features.iloc[:, :5])
        
        # Save the properly extracted clinical data
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        clinical_features.to_csv(out_clinical_data_file)
        print(f"Clinical data saved to {out_clinical_data_file}")
    except Exception as e:
        print(f"Error extracting clinical data: {e}")
        is_trait_available = False
else:
    print("No trait data (Epilepsy) available in this dataset based on previous analysis.")

# 3. Link clinical and genetic data if both are available
if is_trait_available and is_gene_available:
    try:
        # Debug the column names to ensure they match
        print(f"Gene data columns (first 5): {gene_data.columns[:5].tolist()}")
        print(f"Clinical data columns (first 5): {clinical_features.columns[:5].tolist()}")
        
        # Check for common sample IDs
        common_samples = set(gene_data.columns).intersection(clinical_features.columns)
        print(f"Found {len(common_samples)} common samples between gene and clinical data")
        
        if len(common_samples) > 0:
            # Link the clinical and genetic data
            linked_data = geo_link_clinical_genetic_data(clinical_features, gene_data)
            print(f"Initial linked data shape: {linked_data.shape}")
            
            # Debug the trait values before handling missing values
            print("Preview of linked data (first 5 rows, first 5 columns):")
            print(linked_data.iloc[:5, :5])
            
            # Handle missing values
            linked_data = handle_missing_values(linked_data, trait)
            print(f"Linked data shape after handling missing values: {linked_data.shape}")
            
            if linked_data.shape[0] > 0:
                # Check for bias in trait and demographic features
                is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
                
                # Validate the data quality and save cohort info
                note = "Dataset contains gene expression data from GBM cell cultures, but no epilepsy phenotype information."
                is_usable = validate_and_save_cohort_info(
                    is_final=True,
                    cohort=cohort,
                    info_path=json_path,
                    is_gene_available=is_gene_available,
                    is_trait_available=is_trait_available,
                    is_biased=is_biased,
                    df=linked_data,
                    note=note
                )
                
                # Save the linked data if it's usable
                if is_usable:
                    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
                    linked_data.to_csv(out_data_file)
                    print(f"Linked data saved to {out_data_file}")
                else:
                    print("Data not usable for the trait study - not saving final linked data.")
            else:
                print("After handling missing values, no samples remain.")
                validate_and_save_cohort_info(
                    is_final=True,
                    cohort=cohort,
                    info_path=json_path,
                    is_gene_available=is_gene_available,
                    is_trait_available=is_trait_available,
                    is_biased=True,
                    df=pd.DataFrame(),
                    note="No valid samples after handling missing values."
                )
        else:
            print("No common samples found between gene expression and clinical data.")
            validate_and_save_cohort_info(
                is_final=True,
                cohort=cohort,
                info_path=json_path,
                is_gene_available=is_gene_available,
                is_trait_available=is_trait_available,
                is_biased=True,
                df=pd.DataFrame(),
                note="No common samples between gene expression and clinical data."
            )
    except Exception as e:
        print(f"Error linking or processing data: {e}")
        validate_and_save_cohort_info(
            is_final=True,
            cohort=cohort,
            info_path=json_path,
            is_gene_available=is_gene_available,
            is_trait_available=is_trait_available,
            is_biased=True,  # Assume biased if there's an error
            df=pd.DataFrame(),  # Empty dataframe for metadata
            note=f"Error in data processing: {str(e)}"
        )
else:
    # Create an empty DataFrame for metadata purposes
    empty_df = pd.DataFrame()
    
    # We can't proceed with linking if either trait or gene data is missing
    print("Cannot proceed with data linking due to missing trait or gene data.")
    validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=is_gene_available,
        is_trait_available=is_trait_available,
        is_biased=True,  # Data is unusable if we're missing components
        df=empty_df,  # Empty dataframe for metadata
        note="Missing essential data components for linking: dataset contains gene expression data from GBM cell cultures, but no epilepsy phenotype information."
    )

Gene data shape before normalization: (15042, 72)


Gene data shape after normalization: (14516, 72)


Normalized gene data saved to ../../output/preprocess/Epilepsy/gene_data/GSE29796.csv
Extracted clinical data shape: (1, 72)
Preview of clinical data (first 5 samples):
          GSM738329  GSM738330  GSM738331  GSM738332  GSM738333
Epilepsy        1.0        1.0        1.0        1.0        1.0
Clinical data saved to ../../output/preprocess/Epilepsy/clinical_data/GSE29796.csv
Gene data columns (first 5): ['GSM738329', 'GSM738330', 'GSM738331', 'GSM738332', 'GSM738333']
Clinical data columns (first 5): ['GSM738329', 'GSM738330', 'GSM738331', 'GSM738332', 'GSM738333']
Found 72 common samples between gene and clinical data
Initial linked data shape: (72, 14517)
Preview of linked data (first 5 rows, first 5 columns):
           Epilepsy        A2M   A2M-AS1     A2MP1      AACS
GSM738329       1.0   9.518956  5.217719  6.188802  6.216576
GSM738330       1.0   8.460140  5.637911  7.387368  6.109275
GSM738331       1.0   9.631527  4.637884  5.729615  5.782923
GSM738332       1.0   8.104231  

Linked data shape after handling missing values: (72, 14517)
For the feature 'Epilepsy', the least common label is '1.0' with 20 occurrences. This represents 27.78% of the dataset.
The distribution of the feature 'Epilepsy' in this dataset is fine.



Linked data saved to ../../output/preprocess/Epilepsy/GSE29796.csv
